In [ ]:
# @title 1. (FIXED) Clean Install & Download Real_HAT_GAN
import os
import shutil
from google.colab import drive

# 1. Mount Drive
drive.mount('/content/drive')

# 2. Reset Folder (Supaya bersih)
%cd /content
if os.path.exists('HAT'):
    shutil.rmtree('HAT')
    print("Membersihkan instalasi lama...")

# 3. Clone Repository
print("Cloning HAT Repository...")
!git clone https://github.com/XPixelGroup/HAT.git
%cd HAT

# 4. Install Dependencies
!pip install basicsr
!pip install -r requirements.txt
!python setup.py develop

# 5. Download Model (Link Mirror Hugging Face - Acly)
# Kita gunakan model 'Real_HAT_GAN_sharper' yang sangat tajam untuk foto
import os
if not os.path.exists('experiments/pretrained_models'):
    os.makedirs('experiments/pretrained_models')

print("Downloading Model Real_HAT_GAN_sharper.pth...")
!wget https://huggingface.co/Acly/hat/resolve/main/Real_HAT_GAN_sharper.pth -O experiments/pretrained_models/Real_HAT_GAN_sharper.pth

print("\n=== INSTALASI SUKSES! LANJUT KE LANGKAH 2 ===")

In [ ]:
# @title 2. Create Background process to move file
import os
import subprocess

# Define paths based on previous cell's context
output_folder_drive = '/content/drive/MyDrive/hatoutput'
results_root = 'results' # Relative to /content/HAT

# 1. Define the shell script content
mover_script_content = rf"""#!/bin/bash

# --- Configuration for Mover Script ---
OUTPUT_FOLDER_DRIVE="{output_folder_drive}"
RESULTS_ROOT="{results_root}"
MOVER_LOG_FILE="mover_log.log"

# Redirect all subsequent output (stdout and stderr) of this script to the log file
exec > >(tee -a $MOVER_LOG_FILE) 2>&1

echo "$(date): Mover script started."

# Ensure we are in the correct directory for relative paths
# The notebook should already be in /content/HAT, but this adds robustness
if [[ "$(pwd)" != "/content/HAT" ]]; then
    echo "$(date): Changing directory to /content/HAT"
    cd /content/HAT
    if [ $? -ne 0 ]; then
        echo "$(date): Error: Could not change to /content/HAT. Exiting mover script." >&2
        exit 1
    fi
fi

# Create output directory if it doesn't exist
mkdir -p "$OUTPUT_FOLDER_DRIVE"
echo "$(date): Ensured output directory exists: $OUTPUT_FOLDER_DRIVE"

# Use a temporary file to track files that have already been processed in this session.
# This prevents redundant moves and excessive log spam.
MOVED_FILES_TRACKER="/tmp/moved_files_tracker_$(date +%s)_$$.txt"
# Initialize the tracker file as empty
> "$MOVED_FILES_TRACKER"
echo "$(date): Initialized moved files tracker at: $MOVED_FILES_TRACKER"

while true; do
    # echo "$(date): Checking for new upscaled images..."

    # Find image files (.png, .jpg, .jpeg) within any 'visualization' subdirectory under RESULTS_ROOT
    # The -print0 and xargs -0 are for handling filenames with spaces or special characters
    find "$RESULTS_ROOT" -type d -name "visualization" -print0 | xargs -0 -I {{}} find {{}} -type f \( -iname "*.png" -o -iname "*.jpg" -o -iname "*.jpeg" \) -print0 | while IFS= read -r -d $'\0' SRC_PATH; do

        if [ -f "$SRC_PATH" ]; then
            FILE_NAME=$(basename "$SRC_PATH")
            DEST_PATH="$OUTPUT_FOLDER_DRIVE/$FILE_NAME"

            # Get file size to filter out potentially incomplete or empty files
            FILE_SIZE=$(stat -c%s "$SRC_PATH" 2>/dev/null)

            if [ "$FILE_SIZE" -gt 1000 ]; then # Consider files larger than 1KB as valid
                echo "$(date): Detected new image: $SRC_PATH (Size: $FILE_SIZE bytes). Attempting to move..."

                cp "$SRC_PATH" "$DEST_PATH"
                if [ $? -eq 0 ]; then
                    echo "$(date): Successfully moved $FILE_NAME."
                else
                    echo "$(date): Failed to copy $FILE_NAME. Error: $? (mv command exit code)"
                fi
            else
                # echo "$(date): Skipping small file $FILE_NAME (size: $FILE_SIZE bytes). May be incomplete or not an image."
                :
            fi
        fi
    done
    sleep 60
done

# This part should ideally not be reached as the loop runs indefinitely unless killed.
echo "$(date): Mover script finished (unexpectedly)."
"""

# Define the script file name
script_name = "move_upscaled_files.sh"
script_path = os.path.join(os.getcwd(), script_name)

# 2. Write the script content to a file
with open(script_path, "w") as f:
    f.write(mover_script_content)
print(f"Created shell script: {script_path}")

# 3. Make the script executable
!chmod +x "{script_path}"
print(f"Made script executable: {script_path}")

# 4. Execute the script in the background as a detached process
# Ensure previous log file is cleared
mover_log_file = "mover_log.log"
if os.path.exists(mover_log_file):
    os.remove(mover_log_file)

# Using 'setsid' to run the script in a new session, completely detached.
# This makes it robust against the Colab cell stopping or crashing.
# 'bash' is explicitly used for execution.
# `echo $!` captures the PID of the last background command.
command_to_run = f"setsid bash {script_name} > /dev/null 2>&1 & echo $!"

# Execute the command from the current working directory (/content/HAT)
current_dir = os.getcwd()
try:
    # Use subprocess.check_output to get the PID echoed by the shell command
    process_output = subprocess.check_output(command_to_run, shell=True, cwd=current_dir, text=True)
    mover_pid = int(process_output.strip())
    print(f"Started background file mover script (PID: {mover_pid}). Output is logged to: {mover_log_file}")

    # Optionally save PID to a file for easy termination later if needed
    with open("mover_script_pid.txt", "w") as f:
        f.write(str(mover_pid))

except Exception as e:
    print(f"Error starting background mover script: {e}")
    mover_pid = None

In [ ]:
# @title 3. (FIXED V10 - TILE 256 POWER OF 2) Patch, Config & Run
import os
import shutil
import yaml
import basicsr
import re
import torch

# --- 1. KONFIGURASI PENGGUNA ---
input_folder = '/content/drive/MyDrive/hatsource'
output_folder_drive = '/content/drive/MyDrive/hatoutput'
model_path = '/content/HAT/experiments/pretrained_models/Real_HAT_GAN_sharper.pth'

# --- 2. BERSIHKAN GPU & CEK FOLDER ---
torch.cuda.empty_cache()
print("Checking paths...")
if not os.path.exists(input_folder):
    os.makedirs(input_folder)
    print(f"⚠️ Folder {input_folder} baru dibuat. Mohon isi dengan foto dulu!")

# --- 3. SYSTEM PATCH (Wajib Diulang Tiap Restart) ---
print("🛠️ Memperbaiki sistem library AI...")
try:
    basicsr_path = os.path.dirname(basicsr.__file__)

    # A. Patch Torchvision
    deg_file = os.path.join(basicsr_path, 'data', 'degradations.py')
    with open(deg_file, 'r') as f: content = f.read()
    if "from torchvision.transforms.functional_tensor" in content:
        content = content.replace("from torchvision.transforms.functional_tensor", "from torchvision.transforms.functional")
        with open(deg_file, 'w') as f: f.write(content)

    # B. Patch Logger
    opt_file = os.path.join(basicsr_path, 'utils', 'options.py')
    with open(opt_file, 'r') as f: opt_content = f.read()
    bad_code = "msg += ' ' * (indent_level * 2) + k + ': ' + str(v) + '\\n'"
    good_code = "msg += ' ' * (indent_level * 2) + str(k) + ': ' + str(v) + '\\n'"
    if bad_code in opt_content:
        opt_content = opt_content.replace(bad_code, good_code)
        with open(opt_file, 'w') as f: f.write(opt_content)
    print("✅ System Patched.")
except Exception as e: print(f"⚠️ Info Patch: {e}")

# --- 4. CONFIGURATION (TILE 256) ---
print("\n📝 Mengatur Konfigurasi (Tile 256 - The Sweet Spot)...")

base_config_path = 'options/test/HAT_GAN_Real_SRx4.yml'
target_config_path = 'options/test/Auto_Config_V10.yml'

with open(base_config_path, 'r') as f:
    config_str = f.read()

# 1. Path Injection
config_str = re.sub(r'dataroot_lq: .*', f'dataroot_lq: {input_folder}', config_str)
config_str = re.sub(r'pretrain_network_g: .*', f'pretrain_network_g: {model_path}', config_str)

# 2. Architecture Injection (Large Model)
if "num_feat: 64" in config_str:
    config_str = config_str.replace("num_feat: 64", "embed_dim: 180")
    # Inject mlp_ratio: 2 which is critical for Real_HAT_GAN
    config_str = config_str.replace("embed_dim: 180", "embed_dim: 180\n  mlp_ratio: 2")

# 3. Tile Setting (256)
# Kita hapus blok tile lama (jika ada)
config_str = re.sub(r'tile:\s*\n\s*tile_size:\s*\d+', '', config_str)
# Tambah blok baru
config_str += "\n\ntile:\n  tile_size: 256\n  tile_pad: 32\n"

with open(target_config_path, 'w') as f:
    f.write(config_str)

# --- 5. EXECUTE ---
print(f"🚀 Memulai Upscale...")
# Command line override untuk memastikan dataset mode benar
!python hat/test.py -opt options/test/Auto_Config_V10.yml

# --- 6. MOVE FILES ---
print("\n📦 Menyalin hasil...")
results_root = 'results'
count = 0

if not os.path.exists(output_folder_drive):
    os.makedirs(output_folder_drive)

if os.path.exists(results_root):
    for root, dirs, files in os.walk(results_root):
        for file in files:
            # Filter hanya file gambar hasil (biasanya ada _HAT di namanya atau murni gambar)
            if file.lower().endswith(('.png', '.jpg', '.jpeg')) and "visualization" in root:
                src_path = os.path.join(root, file)
                # Cek ukuran file biar bukan thumbnail kosong
                if os.path.getsize(src_path) > 1000:
                    shutil.copy(src_path, output_folder_drive)
                    count += 1

if count > 0:
    print(f"✅ SUKSES! {count} foto berhasil di-upscale.")
    print(f"📂 Cek folder Drive: {output_folder_drive}")
else:
    print("⚠️ Script jalan tapi output belum tersalin. Cek log di atas.")